# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235940220976                   -0.50    7.0
  2   -7.250443104415       -1.84       -1.40    1.0
  3   -7.251286719040       -3.07       -2.13    3.0
  4   -7.251183313775   +   -3.99       -2.06    4.0
  5   -7.251333060098       -3.82       -2.77    2.0
  6   -7.251338191168       -5.29       -3.18    2.0
  7   -7.251338776378       -6.23       -4.01    1.0
  8   -7.251338794761       -7.74       -4.24    4.0
  9   -7.251338798171       -8.47       -4.69    2.0
 10   -7.251338798646       -9.32       -5.17    3.0
 11   -7.251338798701      -10.26       -5.84    3.0
 12   -7.251338798704      -11.50       -6.28    3.0
 13   -7.251338798704      -12.66       -6.75    3.0
 14   -7.251338798705      -13.36       -7.19    3.0
 15   -7.251338798705      -14.57       -7.75    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.09009866765092887
[ Info: Arnoldi iteration step 2: normres = 0.498326410223011
[ Info: Arnoldi iteration step 3: normres = 0.6703650996463831
[ Info: Arnoldi iteration step 4: normres = 0.21944219496087777
[ Info: Arnoldi iteration step 5: normres = 0.6084759403412064
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.21e-02, 8.52e-02, 5.52e-01, 2.40e-01, 2.96e-02)
[ Info: Arnoldi iteration step 6: normres = 0.23023644549492386
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.28e-03, 1.08e-01, 1.62e-01, 9.60e-02, 4.19e-02)
[ Info: Arnoldi iteration step 7: normres = 0.11016143075104032
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.23e-04, 1.23e-02, 1.49e-02, 7.85e-02, 6.58e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13066255782857455
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.14e-